# News Positivity Research

## Intro

This notebook assesses the performance of several pre-trained models for scoring the positivity of news sites:
- [AFINN](https://github.com/fnielsen/afinn)
- [VADER](https://github.com/cjhutto/vaderSentiment)
- [DistilBERT](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
- [roBERTa](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment)
- [BERTweet](https://huggingface.co/cardiffnlp/bertweet-base-sentiment)

AFINN and VADER are two popular dictionary-based models. DistilBERT is the default sentiment analysis model offered by Hugging Face in their [quick tour](https://huggingface.co/transformers/quicktour.html). roBERTa and BERTweet are two of the best performing models in the [TweetEval benchmark study](https://arxiv.org/pdf/2010.12421.pdf).

We're only looking at the landing pages (e.g. https://bbc.co.uk) and not the article pages (e.g. https://www.bbc.co.uk/news/uk-england-birmingham-58282348).

The [test.csv](./test.csv) file contains the test dataset. The text content in the dataset was gathered using the [data.ipynb](./data.ipynb) script, which was run on 3 websites: [bbc.co.uk](https:/bbc.co.uk), [thecanary.co](https://thecanary.co) and [positive.news](https://positive.news). For each text piece, I manually assigned a sentiment score of -1 if it was negative, and +1 if it was neutral or positive.

We start by loading the test dataset into a pandas DataFrame.

In [1]:
import pandas as pd

results = pd.read_csv("test.csv")

Score the dataset with AFINN and VADER

In [2]:
from afinn import Afinn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

afinn = Afinn()
results["afinn"] = results.apply(lambda row: 1 if afinn.score(row["text"]) >= 0 else -1, axis=1)

vader = SentimentIntensityAnalyzer()
results["vader"] = results.apply(lambda row: 1 if vader.polarity_scores(row["text"])["compound"] >= 0 else -1, axis=1)

Score the dataset with DistilBERT.

In [3]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

results["distilbert"] = results.apply(lambda row: -1 if classifier(row["text"])[0]["label"] == "NEGATIVE" else 1, axis=1)

Fetch the roBERTa model.

In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

ROBERTA_MODEL = "cardiffnlp/twitter-roberta-base-sentiment"

roberta_tokenizer = AutoTokenizer.from_pretrained(ROBERTA_MODEL)
roberta_model = AutoModelForSequenceClassification.from_pretrained(ROBERTA_MODEL)

## Un-comment when first run so you can download the model on your local machine.
#roberta_model.save_pretrained(ROBERTA_MODEL)
#roberta_tokenizer.save_pretrained(ROBERTA_MODEL)

Score the dataset with roBERTa. We've defined the `model_output` function as we'll use it when scoring with BERTweet.

In [6]:
import numpy as np
from scipy.special import softmax

def model_output(text_input, model_input, tokenizer_input):
    encoded_input = tokenizer_input(text_input, return_tensors='pt')
    output = model_input(**encoded_input)
    
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    if scores[0] > scores[1] and scores[0] > scores[2]:
        return -1
    else:
        return 1
    
results["roberta"] = results.apply(lambda row: model_output(row["text"], roberta_model, roberta_tokenizer), axis=1)

Fetch the BERTweet model. 

In [12]:
BERTWEET_MODEL = "cardiffnlp/bertweet-base-sentiment"
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_MODEL)
bertweet_model = AutoModelForSequenceClassification.from_pretrained(BERTWEET_MODEL)

## Un-comment when first run so you can download the model on your local machine.
#bertweet_model.save_pretrained(BERTWEET_MODEL)
#bertweet_tokenizer.save_pretrained(BERTWEET_MODEL)

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


Score the dataset with BERTweet

In [14]:
results["bertweet"] = results.apply(lambda row: model_output(row["text"], bertweet_model, bertweet_tokenizer), axis=1)

Assess the accuracy of the models

In [20]:
from sklearn.metrics import accuracy_score

for model in ["afinn","vader","distilbert","roberta","bertweet"]:
    print(model, str(round(accuracy_score(results["sentiment"],results[model])*100,1))+"%")

afinn 75.2%
vader 71.8%
distilbert 65.8%
roberta 79.5%
bertweet 77.8%
